In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm

warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [132]:
def imputeWeather(weather_df, method:str='linear', gap_limit:int=None, limit_direction:str='forward'):
    grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=method, limit=gap_limit, limit_direction=limit_direction))
    
    if 'cloud_coverage' in grouped_weather_df.columns:
        grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
    grouped_weather_df.reset_index(inplace=True)
    return grouped_weather_df.drop('index', axis=1)
        

In [133]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.int32, 'year_built': np.float32, 'floor_count': np.float32},
    'weather' : {'site_id': np.int8, 'air_temperature': np.float32, 'cloud_coverage': np.float32, 'dew_temperature': np.float32,
                     'precip_depth_1_hr': np.float32, 'sea_level_pressure': np.float32, 'wind_direction': np.float32, 'wind_speed': np.float32}
}

file_loc = {}    
for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
    for name in ['building_metadata','weather_train','weather_test','train','test']:
        if path.exists(dir_path + name + '.csv'):
            file_loc[name]= dir_path + name + '.csv'
    
    building = pd.read_csv(file_loc['building_metadata'], dtype=file_dtype['building_metadata'])
    weather_train = pd.read_csv(file_loc['weather_train'], dtype=file_dtype['weather'])
    weather_test = pd.read_csv(file_loc['weather_test'], dtype=file_dtype['weather'])
    train = pd.read_csv(file_loc['train'], dtype=file_dtype['train'])
    test = pd.read_csv(file_loc['test'], dtype=file_dtype['test'])

print(weather_train.isna().sum())    
weather_train = imputeWeather(weather_train)
print(weather_train.isna().sum())
weather_test = imputeWeather(weather_test)
    
train = train.merge(building, on='building_id', how='left')
test = test.merge(building, on='building_id', how='left')
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

del weather_train, weather_test
gc.collect()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64
site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        17257
dew_temperature           0
precip_depth_1_hr     26423
sea_level_pressure     8875
wind_direction            0
wind_speed                0
dtype: int64


181

In [134]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [135]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [136]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [137]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [138]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [139]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        return df.drop(self._drop_cols, axis=1)

    def fit(self, X, y=None, **fit_params):
        return self

In [140]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [141]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [142]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [143]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [144]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [145]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [146]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [147]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [148]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [149]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)



In [150]:

gbm=LGBMRegressor(n_estimators=500, # for accuracy use large numbers like 6000 
                  learning_rate=0.1,
                  feature_fraction=0.9,
                  subsample=0.1,  # batches of 20% of the data
                  subsample_freq=1,
                  num_leaves=160,
                  max_depth=10,
                  metric='rmse',
                  verbose= 100)


In [151]:
# pre_a_pipes is for preprocessing that doesn't change impute
# values
pre_a_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        ('convertToDatetime', ConvertToDatetime()),
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('logSquareFeet', LogSquareFeet()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
    ]
)

train_X = pre_a_pipes.transform(train.drop('meter_reading', axis=1))
test_X = pre_a_pipes.transform(test.drop(['row_id'], axis=1))
train_y = np.log1p(train['meter_reading'])

print(train_X.shape)
print(test_X.shape)
print(train_X.sample(n=20,  random_state=42))

del train, test
gc.collect()

(20216100, 18)
(41697600, 18)
         building_id meter           timestamp site_id  \
14245562        1324     1 2016-09-16 16:00:00      14   
1282718         1013     0 2016-01-24 06:00:00      10   
13883790         229     1 2016-09-10 07:00:00       2   
4781820          217     3 2016-04-01 01:00:00       2   
10415393        1434     0 2016-07-10 04:00:00      15   
1057008         1047     0 2016-01-20 04:00:00      12   
4507399          911     1 2016-03-26 20:00:00       9   
19478829        1039     0 2016-12-18 23:00:00      12   
8955615          265     0 2016-06-14 06:00:00       2   
13799839         896     0 2016-09-08 19:00:00       9   
15647011         973     0 2016-10-11 11:00:00       9   
2524294          813     0 2016-02-16 08:00:00       8   
10016102         870     0 2016-07-03 02:00:00       8   
3915750          898     0 2016-03-15 03:00:00       9   
17217526         903     0 2016-11-08 09:00:00       9   
11478           1442     2 2016-01-01 04:0

352

In [152]:
# pre_b_pipes is for imputed values and final
# drops before modeling
drop_cols = ['timestamp', 'square_feet',
             'wind_direction',
             'precip_depth_1_hr', 
             'year_built' ]
fill_w_mean = ['floor_count','air_temperature','dew_temperature', 
              'precip_depth_1_hr', 'sea_level_pressure']
fill_w_zero = []


pre_b_pipes = Pipeline(
    steps=[
        ('fillMean',FillMean(fill_w_mean)),
        ('fillZeros',FillZeros(fill_w_zero)),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('dropClos', DropCols(drop_cols))
    ]
)

pipe = Pipeline(
    steps=[
        ('pre_b_pipes',pre_b_pipes),
        ('gbm', gbm)]
)



In [153]:
# Cross val testing - can be skipped
scores = cross_val_score(pipe, train_X, train_y, cv=5, 
                        scoring=rmsee_scorer)
print('rmsee scores:\n', scores)


rmsee scores:
 [-1.27741894 -1.22431268 -1.03063807 -1.16863322 -1.43158625]


In [154]:
del scores
gc.collect()

87

In [155]:
# Grid param search - can be skpped
grid_param = {
    'gbm__n_estimators': [500],
    'gbm__subsample': [0.1],
    'gbm__learning_rate': [0.1],
    'gbm__num_leaves': [80, 160],
    'gbm__max_depth': [10]
}

gd_sr = GridSearchCV(pipe,
                     param_grid=grid_param,
                     scoring=rmsee_scorer,
                     cv=3,
                     n_jobs=-1)

gd_sr.fit(train_X, train_y)

best_parameters = gd_sr.best_params_
print(best_parameters)
best_result = gd_sr.best_score_
print(best_result)


{'gbm__learning_rate': 0.1, 'gbm__max_depth': 10, 'gbm__n_estimators': 500, 'gbm__num_leaves': 160, 'gbm__subsample': 0.1}
-1.355260118940074


In [156]:
sorted(gd_sr.cv_results_.keys())
for key in gd_sr.cv_results_.keys():
    print(str(key) + "  " + str(gd_sr.cv_results_[key]))


mean_fit_time  [1037.1145548  1219.01221228 2018.24647729 2310.51012079 3452.98459983
 3756.19839891]
std_fit_time  [31.04935087 17.90739042 32.56600359 10.7339371  58.70547761 28.39781594]
mean_score_time  [ 952.49961909 1381.68596133 2672.66339318 4443.65559832 6902.2206138
 8332.0504059 ]
std_score_time  [ 37.48554732  14.36583429 212.15097066  24.25017769 548.62407352
  93.30981057]
param_gbm__learning_rate  [0.1 0.1 0.1 0.1 0.1 0.1]
param_gbm__max_depth  [10 10 10 10 10 10]
param_gbm__n_estimators  [500 500 1000 1000 2000 2000]
param_gbm__num_leaves  [80 160 80 160 80 160]
param_gbm__subsample  [0.1 0.1 0.1 0.1 0.1 0.1]
params  [{'gbm__learning_rate': 0.1, 'gbm__max_depth': 10, 'gbm__n_estimators': 500, 'gbm__num_leaves': 80, 'gbm__subsample': 0.1}, {'gbm__learning_rate': 0.1, 'gbm__max_depth': 10, 'gbm__n_estimators': 500, 'gbm__num_leaves': 160, 'gbm__subsample': 0.1}, {'gbm__learning_rate': 0.1, 'gbm__max_depth': 10, 'gbm__n_estimators': 1000, 'gbm__num_leaves': 80, 'gbm__subsa

In [157]:
del gd_sr
gc.collect

<function gc.collect(generation=2)>

In [158]:
# fit on all the data
pipe.fit(train_X, train_y, 
         gbm__eval_metric=rmsee, gbm__verbose=100)


Pipeline(memory=None,
         steps=[('pre_b_pipes',
                 Pipeline(memory=None,
                          steps=[('fillMean',
                                  <__main__.FillMean object at 0x000001F344830550>),
                                 ('fillZeros',
                                  <__main__.FillZeros object at 0x000001F34475E240>),
                                 ('imputeCloudCoverage',
                                  <__main__.ImputeCloudCoverage object at 0x000001F34475EEF0>),
                                 ('imputeYearBuilt',
                                  <__main__.ImputeYearBuilt object at 0x000001F3675A26A0>),
                                 ('dro...
                               colsample_bytree=1.0, feature_fraction=0.9,
                               importance_type='split', learning_rate=0.1,
                               max_depth=10, metric='rmse',
                               min_child_samples=20, min_child_weight=0.001,
                

In [159]:
# Get features list for fit - can be skipped
imprtc_df = pd.DataFrame()
imprtc_df['feature'] = pre_b_pipes.transform(train_X).columns   
imprtc_df['importance'] = pipe.named_steps['gbm'].feature_importances_
imprtc_df.sort_values('importance', ascending=False, inplace= True)
print(imprtc_df)
print(imprtc_df.nsmallest(7,'importance')['feature'].tolist())



               feature  importance
0          building_id       19164
7      dew_temperature        9981
5      air_temperature        9688
12                hour        9598
8   sea_level_pressure        9286
1                meter        7346
9           wind_speed        3567
11           dayofweek        3106
6       cloud_coverage        2718
10     log_square_feet        2439
2              site_id         876
13        building_age         730
4          floor_count         698
3          primary_use         303
['primary_use', 'floor_count', 'building_age', 'site_id', 'log_square_feet', 'cloud_coverage', 'dayofweek']


In [ ]:
set_size = len(test_X)
iterations = 100
batch_size = set_size // iterations

print(set_size, iterations, batch_size)
assert set_size == iterations * batch_size

meter_reading = []
for i in tqdm(range(iterations)):
    pos = i*batch_size
    batch = np.expm1(pipe.predict(test_X.iloc[pos : pos+batch_size]).clip(0))
    meter_reading.extend(batch)

print(len(meter_reading))
assert len(meter_reading) == set_size

41697600 100 416976


 93%|███████████████████████████████████████████████████████████████████████████▎     | 93/100 [14:12<01:20, 11.47s/it]

In [ ]:
sub = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
# hack to prevent negative numbers
print(sub.shape)
sub['meter_reading'] = meter_reading
sub.to_csv('submission.csv', index = False)